# Fizz Buzz in Tensorflow 
## [@joelgrus](http://twitter.com/joelgrus)
### http://github.com/joelgrus

------------------------


**interviewer:** Welcome, can I get you coffee or anything? Do you need a break?

**me:** No, I've probably had too much coffee already!

**interviewer:** Great, great. And are you OK with writing code on the whiteboard?

**me:** It's the only way I code!

**interviewer:** ...

**me:** That was a joke.

**interviewer:** OK, so are you familiar with "fizz buzz"?

**me:** ...

**interviewer:** Is that a yes or a no?

**me:** It's more of a "I can't believe you're asking me that."

**interviewer:** OK, so I need you to print the numbers from 1 to 100, except that if the number is divisible by 3 print "fizz", if it's divisible by 5 print "buzz", and if it's divisible by 15 print "fizzbuzz".

**me:** I'm familiar with it.

**interviewer:** Great, we find that candidates who can't get this right don't do well here.

**me:** ...

**interviewer:** Here's a marker and an eraser.

**me:** [thinks for a couple of minutes]

**interviewer:** Do you need help getting started?

**me:** No, no, I'm good. So let's start with some standard imports:

In [1]:
import numpy as np
import tensorflow as tf

**interviewer**: Um, you understand the problem is *fizzbuzz*, right?

**me:** Do I ever. So, now let's talk models. I'm thinking a simple multi-layer-perceptron with one hidden layer.

**interviewer:** Perceptron?

**me:** Or neural network, whatever you want to call it. We want the input to be a number, and the output to be the correct "fizzbuzz" representation of that number. In particular, we need to turn each input into a vector of "activations". One simple way would be to convert it to binary.

**interviewer:** Binary?

**me:** Yeah, you know, 0's and 1's? Something like:



In [2]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

**interviewer:** [stares at whiteboard for a minute]

**me:** And our output will be a one-hot encoding of the fizzbuzz representation of the number, where the first position indicates "print as-is", the second indicates "fizz", and so on:

In [3]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])

**interviewer:** OK, that's probably enough.

**me:** That's enough setup, you're exactly right. Now we need to generate some training data. It would be cheating to use the numbers 1 to 100 in our training data, so let's train it on all the remaining numbers up to 1024:

In [4]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])


**interviewer:** ...

**me:** Now we need to set up our model in tensorflow. Off the top of my head I'm not sure how many hidden units to use, maybe 10?

**interviewer:** ...

**me:** Yeah, possibly 100 is better. We can always change it later.

In [5]:
NUM_HIDDEN = 100


We'll need an input variable with width NUM_DIGITS, and an output variable with width 4:



In [6]:
X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

**interviewer:** How far are you intending to take this?

**me:** Oh, just two layers deep -- one hidden layer and one output layer. Let's use randomly-initialized weights for our neurons:

In [7]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

And we're ready to define the model. As I said before, one hidden layer, and let's use, I don't know, ReLU activation:

In [8]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

We can use softmax cross-entropy as our cost function and try to minimize it:



In [9]:
py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

**interviewer:** ...

**me:** And, of course, the prediction will just be the largest output:

In [10]:
predict_op = tf.argmax(py_x, 1)


**interviewer:** Before you get too far astray, the problem you're supposed to be solving is to generate fizz buzz for the numbers from 1 to 100.

**me:** Oh, great point, the predict_op function will output a number from 0 to 3, but we want a "fizz buzz" output:

In [11]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

**interviewer:** ...

**me:** So now we're ready to train the model. Let's grab a tensorflow session and initialize the variables:

Now let's run, say, 1000 epochs of training?

**interviewer:** ...

**me:** Yeah, maybe that's not enough -- so let's do 10000 just to be safe.


In [12]:
NUM_EPOCHS = 10000

And our training data are sequential, which I don't like, so let's shuffle them each iteration:


And each epoch we'll train in batches of, I don't know, 128 inputs?




In [13]:
BATCH_SIZE = 128


So each training pass looks like



and then we can print the accuracy on the training data, since why not?


interviewer: Are you serious?

me: Yeah, I find it helpful to see how the training accuracy evolves.

interviewer: ...

In [14]:
sess = tf.Session()

with sess.as_default():
    tf.global_variables_initializer().run()
    
    for epoch in range(NUM_EPOCHS):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
            
        accuracy = np.mean(np.argmax(trY, axis=1) ==
                           sess.run(predict_op, feed_dict={X: trX, Y: trY}))
        print(epoch, accuracy)

0 0.507042253521
1 0.534127843987
2 0.534127843987
3 0.534127843987
4 0.534127843987
5 0.534127843987
6 0.534127843987
7 0.534127843987
8 0.534127843987
9 0.534127843987
10 0.534127843987
11 0.534127843987
12 0.534127843987
13 0.534127843987
14 0.534127843987
15 0.534127843987
16 0.534127843987
17 0.534127843987
18 0.534127843987
19 0.534127843987
20 0.534127843987
21 0.534127843987
22 0.534127843987
23 0.534127843987
24 0.534127843987
25 0.534127843987
26 0.534127843987
27 0.534127843987
28 0.534127843987
29 0.534127843987
30 0.534127843987
31 0.534127843987
32 0.534127843987
33 0.534127843987
34 0.534127843987
35 0.534127843987
36 0.534127843987
37 0.534127843987
38 0.534127843987
39 0.534127843987
40 0.534127843987
41 0.534127843987
42 0.534127843987
43 0.534127843987
44 0.534127843987
45 0.534127843987
46 0.534127843987
47 0.534127843987
48 0.534127843987
49 0.534127843987
50 0.534127843987
51 0.534127843987
52 0.534127843987
53 0.534127843987
54 0.534127843987
55 0.534127843987
56

440 0.534127843987
441 0.534127843987
442 0.534127843987
443 0.534127843987
444 0.534127843987
445 0.534127843987
446 0.534127843987
447 0.534127843987
448 0.534127843987
449 0.534127843987
450 0.534127843987
451 0.534127843987
452 0.534127843987
453 0.534127843987
454 0.534127843987
455 0.534127843987
456 0.534127843987
457 0.534127843987
458 0.534127843987
459 0.534127843987
460 0.534127843987
461 0.534127843987
462 0.534127843987
463 0.534127843987
464 0.534127843987
465 0.534127843987
466 0.534127843987
467 0.534127843987
468 0.534127843987
469 0.534127843987
470 0.534127843987
471 0.534127843987
472 0.534127843987
473 0.534127843987
474 0.534127843987
475 0.534127843987
476 0.534127843987
477 0.534127843987
478 0.534127843987
479 0.534127843987
480 0.534127843987
481 0.534127843987
482 0.534127843987
483 0.534127843987
484 0.534127843987
485 0.534127843987
486 0.534127843987
487 0.534127843987
488 0.534127843987
489 0.534127843987
490 0.534127843987
491 0.534127843987
492 0.534127

910 0.530877573131
911 0.53196099675
912 0.533044420368
913 0.533044420368
914 0.53196099675
915 0.533044420368
916 0.530877573131
917 0.53196099675
918 0.53196099675
919 0.530877573131
920 0.53196099675
921 0.530877573131
922 0.53196099675
923 0.53196099675
924 0.530877573131
925 0.530877573131
926 0.530877573131
927 0.530877573131
928 0.53196099675
929 0.530877573131
930 0.530877573131
931 0.530877573131
932 0.530877573131
933 0.53196099675
934 0.53196099675
935 0.530877573131
936 0.533044420368
937 0.533044420368
938 0.53196099675
939 0.533044420368
940 0.530877573131
941 0.533044420368
942 0.533044420368
943 0.533044420368
944 0.535211267606
945 0.53196099675
946 0.534127843987
947 0.535211267606
948 0.535211267606
949 0.535211267606
950 0.536294691224
951 0.535211267606
952 0.535211267606
953 0.536294691224
954 0.536294691224
955 0.536294691224
956 0.535211267606
957 0.536294691224
958 0.536294691224
959 0.536294691224
960 0.536294691224
961 0.536294691224
962 0.536294691224
963 0

1341 0.660888407367
1342 0.669555796316
1343 0.639219934995
1344 0.641386782232
1345 0.665222101842
1346 0.646803900325
1347 0.65872156013
1348 0.664138678223
1349 0.656554712893
1350 0.652221018418
1351 0.66630552546
1352 0.664138678223
1353 0.65872156013
1354 0.667388949079
1355 0.645720476706
1356 0.629469122427
1357 0.624052004334
1358 0.656554712893
1359 0.661971830986
1360 0.665222101842
1361 0.65872156013
1362 0.654387865655
1363 0.670639219935
1364 0.639219934995
1365 0.66630552546
1366 0.654387865655
1367 0.648970747562
1368 0.673889490791
1369 0.672806067172
1370 0.673889490791
1371 0.669555796316
1372 0.670639219935
1373 0.672806067172
1374 0.672806067172
1375 0.669555796316
1376 0.65872156013
1377 0.637053087757
1378 0.659804983749
1379 0.669555796316
1380 0.665222101842
1381 0.660888407367
1382 0.661971830986
1383 0.664138678223
1384 0.650054171181
1385 0.668472372698
1386 0.637053087757
1387 0.667388949079
1388 0.66630552546
1389 0.668472372698
1390 0.672806067172
1391 0.

1797 0.827735644637
1798 0.819068255688
1799 0.8255687974
1800 0.829902491874
1801 0.835319609967
1802 0.828819068256
1803 0.826652221018
1804 0.810400866739
1805 0.821235102925
1806 0.834236186349
1807 0.819068255688
1808 0.836403033586
1809 0.838569880823
1810 0.803900325027
1811 0.820151679307
1812 0.827735644637
1813 0.811484290358
1814 0.817984832069
1815 0.827735644637
1816 0.828819068256
1817 0.823401950163
1818 0.835319609967
1819 0.836403033586
1820 0.812567713976
1821 0.800650054171
1822 0.819068255688
1823 0.836403033586
1824 0.841820151679
1825 0.821235102925
1826 0.828819068256
1827 0.827735644637
1828 0.821235102925
1829 0.830985915493
1830 0.822318526544
1831 0.8255687974
1832 0.835319609967
1833 0.821235102925
1834 0.836403033586
1835 0.838569880823
1836 0.827735644637
1837 0.848320693391
1838 0.835319609967
1839 0.829902491874
1840 0.841820151679
1841 0.838569880823
1842 0.840736728061
1843 0.838569880823
1844 0.832069339112
1845 0.836403033586
1846 0.842903575298
1847

...

6412 0.997833152763
6413 1.0
6414 0.998916576381
6415 0.998916576381
6416 1.0
6417 1.0
6418 0.998916576381
6419 0.998916576381
6420 1.0
6421 1.0
6422 1.0
6423 1.0
6424 0.998916576381
6425 1.0
6426 1.0
6427 1.0
6428 1.0
6429 1.0
6430 1.0
6431 0.989165763814
6432 1.0
6433 1.0
6434 1.0
6435 1.0
6436 1.0
6437 1.0
6438 1.0
6439 1.0
6440 1.0
6441 1.0
6442 1.0
6443 0.998916576381
6444 1.0
6445 1.0
6446 0.998916576381
6447 1.0
6448 1.0
6449 1.0
6450 1.0
6451 1.0
6452 0.997833152763
6453 0.997833152763
6454 0.998916576381
6455 1.0
6456 1.0
6457 0.997833152763
6458 1.0
6459 1.0
6460 1.0
6461 0.997833152763
6462 1.0
6463 1.0
6464 0.998916576381
6465 1.0
6466 1.0
6467 0.998916576381
6468 1.0
6469 1.0
6470 0.998916576381
6471 1.0
6472 1.0
6473 0.997833152763
6474 1.0
6475 1.0
6476 1.0
6477 1.0
6478 0.997833152763
6479 0.998916576381
6480 0.998916576381
6481 1.0
6482 1.0
6483 1.0
6484 0.998916576381
6485 1.0
6486 0.998916576381
6487 0.997833152763
6488 1.0
6489 1.0
6490 1.0
6491 0.998916576381
6492 

7101 1.0
7102 1.0
7103 1.0
7104 0.998916576381
7105 1.0
7106 1.0
7107 1.0
7108 1.0
7109 1.0
7110 0.998916576381
7111 1.0
7112 1.0
7113 1.0
7114 1.0
7115 0.998916576381
7116 1.0
7117 0.998916576381
7118 1.0
7119 1.0
7120 1.0
7121 0.998916576381
7122 1.0
7123 0.998916576381
7124 1.0
7125 0.984832069339
7126 1.0
7127 1.0
7128 1.0
7129 1.0
7130 1.0
7131 1.0
7132 1.0
7133 1.0
7134 1.0
7135 1.0
7136 1.0
7137 1.0
7138 1.0
7139 1.0
7140 1.0
7141 1.0
7142 1.0
7143 0.984832069339
7144 1.0
7145 0.998916576381
7146 1.0
7147 1.0
7148 1.0
7149 0.998916576381
7150 1.0
7151 0.998916576381
7152 0.998916576381
7153 0.998916576381
7154 1.0
7155 0.998916576381
7156 1.0
7157 1.0
7158 1.0
7159 1.0
7160 1.0
7161 1.0
7162 1.0
7163 1.0
7164 1.0
7165 1.0
7166 1.0
7167 1.0
7168 1.0
7169 0.998916576381
7170 1.0
7171 1.0
7172 1.0
7173 1.0
7174 1.0
7175 1.0
7176 1.0
7177 0.998916576381
7178 1.0
7179 1.0
7180 1.0
7181 1.0
7182 1.0
7183 1.0
7184 1.0
7185 1.0
7186 0.998916576381
7187 1.0
7188 1.0
7189 1.0
7190 1.0
719

7932 1.0
7933 1.0
7934 1.0
7935 1.0
7936 1.0
7937 1.0
7938 1.0
7939 1.0
7940 1.0
7941 1.0
7942 1.0
7943 1.0
7944 1.0
7945 1.0
7946 1.0
7947 1.0
7948 1.0
7949 1.0
7950 1.0
7951 1.0
7952 1.0
7953 1.0
7954 1.0
7955 1.0
7956 1.0
7957 1.0
7958 1.0
7959 1.0
7960 1.0
7961 1.0
7962 0.998916576381
7963 1.0
7964 1.0
7965 1.0
7966 1.0
7967 1.0
7968 1.0
7969 1.0
7970 1.0
7971 1.0
7972 1.0
7973 1.0
7974 1.0
7975 1.0
7976 1.0
7977 1.0
7978 1.0
7979 1.0
7980 1.0
7981 1.0
7982 1.0
7983 1.0
7984 1.0
7985 1.0
7986 1.0
7987 1.0
7988 1.0
7989 1.0
7990 1.0
7991 1.0
7992 1.0
7993 1.0
7994 1.0
7995 1.0
7996 1.0
7997 1.0
7998 1.0
7999 1.0
8000 1.0
8001 1.0
8002 1.0
8003 1.0
8004 1.0
8005 1.0
8006 1.0
8007 1.0
8008 1.0
8009 1.0
8010 1.0
8011 1.0
8012 1.0
8013 1.0
8014 1.0
8015 1.0
8016 1.0
8017 1.0
8018 1.0
8019 1.0
8020 1.0
8021 1.0
8022 1.0
8023 1.0
8024 1.0
8025 1.0
8026 1.0
8027 1.0
8028 1.0
8029 1.0
8030 1.0
8031 1.0
8032 1.0
8033 1.0
8034 1.0
8035 1.0
8036 1.0
8037 1.0
8038 1.0
8039 1.0
8040 1.0
8041 0.9

8865 1.0
8866 1.0
8867 1.0
8868 1.0
8869 1.0
8870 1.0
8871 1.0
8872 1.0
8873 1.0
8874 1.0
8875 1.0
8876 1.0
8877 1.0
8878 1.0
8879 1.0
8880 1.0
8881 1.0
8882 1.0
8883 1.0
8884 1.0
8885 1.0
8886 1.0
8887 1.0
8888 1.0
8889 1.0
8890 1.0
8891 1.0
8892 1.0
8893 1.0
8894 1.0
8895 1.0
8896 1.0
8897 1.0
8898 1.0
8899 1.0
8900 1.0
8901 1.0
8902 1.0
8903 1.0
8904 1.0
8905 1.0
8906 1.0
8907 1.0
8908 1.0
8909 1.0
8910 1.0
8911 1.0
8912 1.0
8913 1.0
8914 1.0
8915 1.0
8916 1.0
8917 1.0
8918 1.0
8919 1.0
8920 1.0
8921 1.0
8922 1.0
8923 1.0
8924 1.0
8925 1.0
8926 1.0
8927 1.0
8928 1.0
8929 1.0
8930 1.0
8931 1.0
8932 1.0
8933 1.0
8934 1.0
8935 1.0
8936 1.0
8937 1.0
8938 1.0
8939 1.0
8940 1.0
8941 1.0
8942 1.0
8943 1.0
8944 1.0
8945 1.0
8946 1.0
8947 1.0
8948 1.0
8949 1.0
8950 1.0
8951 1.0
8952 1.0
8953 1.0
8954 1.0
8955 1.0
8956 1.0
8957 1.0
8958 1.0
8959 1.0
8960 1.0
8961 1.0
8962 1.0
8963 1.0
8964 1.0
8965 1.0
8966 1.0
8967 1.0
8968 1.0
8969 1.0
8970 1.0
8971 1.0
8972 1.0
8973 1.0
8974 1.0
8975 1.0
8

9782 1.0
9783 1.0
9784 1.0
9785 1.0
9786 1.0
9787 1.0
9788 1.0
9789 1.0
9790 1.0
9791 1.0
9792 1.0
9793 1.0
9794 1.0
9795 1.0
9796 1.0
9797 1.0
9798 1.0
9799 1.0
9800 1.0
9801 1.0
9802 1.0
9803 1.0
9804 1.0
9805 1.0
9806 1.0
9807 1.0
9808 1.0
9809 1.0
9810 1.0
9811 1.0
9812 1.0
9813 1.0
9814 1.0
9815 1.0
9816 1.0
9817 1.0
9818 1.0
9819 1.0
9820 1.0
9821 1.0
9822 1.0
9823 1.0
9824 1.0
9825 1.0
9826 1.0
9827 1.0
9828 1.0
9829 1.0
9830 1.0
9831 1.0
9832 1.0
9833 1.0
9834 1.0
9835 1.0
9836 1.0
9837 1.0
9838 1.0
9839 1.0
9840 1.0
9841 1.0
9842 1.0
9843 1.0
9844 1.0
9845 1.0
9846 1.0
9847 1.0
9848 1.0
9849 1.0
9850 1.0
9851 1.0
9852 1.0
9853 1.0
9854 1.0
9855 1.0
9856 1.0
9857 1.0
9858 1.0
9859 1.0
9860 1.0
9861 1.0
9862 1.0
9863 1.0
9864 1.0
9865 1.0
9866 1.0
9867 1.0
9868 1.0
9869 1.0
9870 1.0
9871 1.0
9872 1.0
9873 1.0
9874 1.0
9875 1.0
9876 1.0
9877 1.0
9878 1.0
9879 1.0
9880 1.0
9881 1.0
9882 1.0
9883 1.0
9884 1.0
9885 1.0
9886 1.0
9887 1.0
9888 1.0
9889 1.0
9890 1.0
9891 1.0
9892 1.0
9

**me:** So, once the model has been trained, it's fizz buzz time. Our input should just be the binary encoding of the numbers 1 to 100:

In [15]:
numbers = np.arange(1, 101)
teX = np.transpose(binary_encode(numbers, NUM_DIGITS))

And then our output is just our `fizz_buzz` function applied to the model output:



In [16]:
teY = sess.run(predict_op, feed_dict={X: teX})
output = np.vectorize(fizz_buzz)(numbers, teY)

**interviewer:** ...

**me:** And that should be your fizz buzz!

**interviewer:** Really, that's enough. We'll be in touch.

**me:** In touch, that sounds promising.

**interviewer: ...**

## Postscript

I didn't get the job. It turns out it got some of the outputs wrong! Thanks a lot, machine learning.

In [17]:
actuals = [fizz_buzz(i, fizz_buzz_encode(i).argmax()) for i in numbers]

for i, (predicted, actual) in enumerate(zip(output, actuals)):
    if predicted != actual:
        print("{0} {1} {2}".format(i+1, predicted, actual))

4 fizz 4
23 fizz 23
49 fizz 49
53 fizz 53


I guess maybe I should have used a deeper network.